In [1]:
%reload_ext lab_black
from worstcase import param, derive, unit

* NeoPixel: [WS2812B-2020](https://www.mouser.com/pdfDocs/WS2812B-2020_V10_EN_181106150240761.pdf)
* Charge Pump: [HX4002](https://datasheet.lcsc.com/lcsc/1806121849_HEXIN-HX4002-MFC_C172354.pdf)
* Schottky Diode: [1N5819WS](https://datasheet.lcsc.com/lcsc/1912111437_KEXIN-1N5819WS_C369937.pdf)
* Linear Regulator: [XC6206](https://datasheet.lcsc.com/lcsc/1809201513_Torex-Semicon-XC6206P332MR_C5446.pdf)
* Switching Diode: [1N4148WS](https://datasheet.lcsc.com/lcsc/2109061630_YONGYUTAI-1N4148WS_C2892572.pdf)
* QSPI Flase: [W25Q128JV](https://datasheet.lcsc.com/lcsc/1811142111_Winbond-Elec-W25Q128JVSIQ_C97521.pdf)

In [2]:
# 5V load is assumed to be in the range of 10mA to 100mA
# USB specification says 5V output has 5% tolerance
VOUT_HX4002 = param.bytol(nom=5 * unit.V, tol=200 * unit.mV, rel=False)
VFWD_1N5819WS_HX4002 = param.bytol(nom=250 * unit.mV, tol=50 * unit.mV, rel=False)
VOUT_USB = param.bytol(nom=5 * unit.V, tol=0.05, rel=True)
VFWD_1N5819WS_USB = param.bytol(nom=250 * unit.mV, tol=50 * unit.mV, rel=False)

In [3]:
@derive.byev(VOUT_HX4002, VFWD_1N5819WS_HX4002, VOUT_USB, VFWD_1N5819WS_USB)
def VOUT_5V(hx4002, hx4002_drop, usb, usb_drop):
    return min(hx4002 - hx4002_drop, usb - usb_drop)

In [4]:
# verify the NeoPixel supply voltage requirement
assert VOUT_5V.ub < 5.5 * unit.V
assert VOUT_5V.lb > 4.5 * unit.V, "NeoPixel undervoltage by {:0.2f}.".format(4.5 * unit.V - VOUT_5V.lb)

AssertionError: NeoPixel undervoltage by 0.05 volt.

In [5]:
# 3V3 load is assumed to be in the range of 10mA to 100mA
ILOAD = param.byrange(nom=30 * unit.mA, lb=10 * unit.mA, ub=100 * unit.mA)
VOUT_XC6206 = param.bytol(nom=3.3 * unit.V, tol=0.02, rel=True)
VLOADREG_XC6206 = param.bytol(nom=0 * unit.V, tol=60 * unit.mV, rel=False)
VLINEREG_XC6206 = param.byrange(nom=0.0005, lb=0.0005, ub=0.0025)
VDO_XC6206 = param.byrange(nom=250 * unit.mV, lb=75 * unit.mV, ub=680 * unit.mV)

In [6]:
@derive.byev(VOUT_XC6206, VLOADREG_XC6206, VLINEREG_XC6206, VOUT_5V)
def VOUT_3V3(out, loadreg, linereg, line):
    return out + loadreg + linereg * line

In [7]:
# verify the qspi flash and rp2040 voltage requirements
assert VOUT_3V3.lb > 3.135 * unit.V
assert VOUT_3V3.ub < 3.6 * unit.V

In [8]:
@derive.byev(VOUT_5V, VOUT_3V3, ILOAD, VDO_XC6206)
def LDO_PWR_DISSIPATION(line, out, load, dropout):
    res = 2.2 * unit.ohm
    vin = line - res * load
    assert res * load ** 2 < 100 * unit.mW  # verify series resistor dissipation
    assert vin - out > dropout  # verify regulator dropout is exceeded
    return (vin - out) * load

In [9]:
# verify the linear regulator power dissipation
assert LDO_PWR_DISSIPATION.ub < 250 * unit.mW

In [10]:
VOL_RP2040 = param.byrange(nom=0 * unit.V, lb=0 * unit.V, ub=0.5 * unit.V)
VFWD_1N4148WS = param.bytol(nom=450 * unit.mV, tol=50 * unit.mV, rel=False)

In [11]:
@derive.byev(VOUT_5V, VFWD_1N4148WS, VOUT_3V3)
def NEOPIX_VIH_MARGIN(vcc, vfwd, vdd):
    return vdd + vfwd - 0.7 * vcc

In [12]:
@derive.byev(VOUT_5V, VFWD_1N4148WS, VOL_RP2040)
def NEOPIX_VIL_MARGIN(vcc, vfwd, vol):
    return 0.3 * vcc - (vol + vfwd)

In [13]:
# verify the NeoPixel input digital voltage level
assert NEOPIX_VIH_MARGIN.lb > 0 * unit.V
assert NEOPIX_VIL_MARGIN.lb > 0 * unit.V